In [15]:
#%load_ext nb_black

In [16]:
import os
import io
from datetime import datetime
import logging

In [17]:
import boto3
import sagemaker
from sagemaker.session import TrainingInput
from sagemaker import image_uris
from sagemaker import hyperparameters

In [18]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

In [19]:
region = sagemaker.Session().boto_region_name
print(region)

us-east-1


In [20]:
# role_arn = sagemaker.get_execution_role()
#role_arn = os.getenv("SGMKR_ROLE_ARN")
role_arn = "arn:aws:iam::012345678901:role/service-role/AmazonSageMaker-ExecutionRole-20240718T104942"

In [21]:
bucket = "pat-demo-bkt"
prefix = "iris"

In [22]:
!aws s3 ls {bucket}/{prefix}/

                           PRE batch_transform/
                           PRE data/
                           PRE model/
2024-10-16 17:58:20          0 


In [23]:
!aws s3 ls {bucket}/{prefix}/data/ --recursive

2024-10-16 17:58:39          0 iris/data/
2024-10-16 17:59:10        950 iris/data/iris_test.csv
2024-10-16 17:59:07       1900 iris/data/iris_train.csv


In [24]:
train_file = "data/iris_train.csv"
valid_file = "data/iris_test.csv"

In [25]:
train_ip = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, train_file), content_type="csv"
)
valid_ip = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, valid_file), content_type="csv"
)

In [26]:
model_op = "s3://{}/{}/{}".format(bucket, prefix, "model")

In [27]:
train_image_uri = sagemaker.image_uris.retrieve("xgboost", region, "latest")
print(train_image_uri)

811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest


In [28]:
base_job_name = "iris-xgboost"

In [29]:
xgb_estimator = sagemaker.estimator.Estimator(
    image_uri=train_image_uri,
    role=role_arn,
    base_job_name=base_job_name,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    volume_size=5,
    output_path=model_op,
    sagemaker_session=sagemaker.Session(),
)

In [30]:
xgb_estimator.set_hyperparameters(
    num_class=3, max_depth=5, num_round=10, objective="multi:softmax",
)

In [31]:
# xgb_estimator.set_hyperparameters(
#     num_class=3,
#     max_depth=5,
#     eta=0.2,
#     gamma=4,
#     min_child_weight=6,
#     subsample=0.7,
#     objective="multi:softmax",
#     num_round=10,
# )

In [32]:
job_name = "iris-xgboost-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(job_name)

iris-xgboost-2024-10-17-17-03-48


In [33]:
xgb_estimator.fit(
    {"train": train_ip, "validation": valid_ip}, wait=True, job_name=job_name
)

INFO:sagemaker:Creating training-job with name: iris-xgboost-2024-10-17-17-03-48


2024-10-18 00:03:52 Starting - Starting the training job...
2024-10-18 00:04:17 Starting - Preparing the instances for training......
2024-10-18 00:05:28 Downloading - Downloading the training image......
2024-10-18 00:06:39 Training - Training image download completed. Training in progress...Arguments: train
[2024-10-18:00:06:50:INFO] Running standalone xgboost training.
[2024-10-18:00:06:50:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8463.78mb
[2024-10-18:00:06:50:INFO] Determined delimiter of CSV input is ','
[00:06:50] S3DistributionType set as FullyReplicated
[00:06:50] 100x4 matrix with 400 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-10-18:00:06:50:INFO] Determined delimiter of CSV input is ','
[00:06:50] S3DistributionType set as FullyReplicated
[00:06:50] 50x4 matrix with 200 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[00:06:50] src/tree/upda

In [34]:
!aws s3 ls {bucket}/{prefix}/model/

                           PRE iris-xgboost-2024-10-17-13-45-55/
                           PRE iris-xgboost-2024-10-17-17-03-48/


### Inference 

In [37]:
from sagemaker.serializers import CSVSerializer

#### Deploy the model as an endpoint

In [38]:
type(xgb_estimator)

sagemaker.estimator.Estimator

In [39]:
xgb_predictor = xgb_estimator.deploy(
    initial_instance_count=1, instance_type="ml.t2.medium", serializer=CSVSerializer()
)

INFO:sagemaker:Creating model with name: iris-xgboost-2024-10-18-00-19-37-448
INFO:sagemaker:Creating endpoint-config with name iris-xgboost-2024-10-18-00-19-37-448
INFO:sagemaker:Creating endpoint with name iris-xgboost-2024-10-18-00-19-37-448


------------!

#### Predictor single record

In [40]:
xgb_predictor.predict("7.7, 3.0, 6.1, 2.3")

b'2.0'

#### Endpoint

In [41]:
endpoint_name = xgb_predictor.endpoint_name
print(endpoint_name)

iris-xgboost-2024-10-18-00-19-37-448


In [43]:
sgmkr_runtime = boto3.client("runtime.sagemaker")

#### Endpoint - One record

In [48]:
payload_csv_text = "7.7, 3.0, 6.1, 2.3"
response = sgmkr_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text
)
print(response)

{'ResponseMetadata': {'RequestId': '73272927-f70f-4745-961d-465904399785', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '73272927-f70f-4745-961d-465904399785', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Fri, 18 Oct 2024 00:40:39 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '3', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x0000019EB5B5F5B0>}


In [49]:
print(response["Body"].read().decode())

2.0


#### Endpoint - Multiple records

In [50]:
payload_csv_text = "7.7, 3.0, 6.1, 2.3 \n 7.9, 3.8, 6.4, 2.1"

response = sgmkr_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text
)
print(response["Body"].read().decode())

2.0,2.0


#### Endpoint - Multiple records from a local file

In [51]:
csv_buffer = open("data/iris_infer.csv")
payload_csv_text = csv_buffer.read()

response = sgmkr_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text
)
print(response["Body"].read().decode())

2.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,2.0


In [52]:
payload_csv_text

'6.3,2.8,5.1,1.5\n6.3,3.3,4.7,1.6\n5.0,3.4,1.5,0.2\n5.8,2.7,4.1,1.0\n7.3,2.9,6.3,1.8\n4.9,2.4,3.3,1.0\n5.7,2.8,4.5,1.3\n5.7,3.8,1.7,0.3\n5.6,3.0,4.5,1.5\n5.5,2.3,4.0,1.3\n4.4,3.2,1.3,0.2\n5.8,4.0,1.2,0.2\n5.1,3.3,1.7,0.5\n5.1,3.4,1.5,0.2\n5.4,3.7,1.5,0.2\n6.4,2.8,5.6,2.2\n6.0,3.0,4.8,1.8\n5.6,2.5,3.9,1.1\n7.7,2.8,6.7,2.0\n5.7,2.8,4.1,1.3\n6.5,3.0,5.2,2.0\n5.6,3.0,4.1,1.3\n4.7,3.2,1.3,0.2\n6.5,3.0,5.5,1.8\n4.6,3.6,1.0,0.2\n6.5,3.0,5.8,2.2\n6.7,3.1,5.6,2.4\n5.0,3.2,1.2,0.2\n5.4,3.4,1.7,0.2\n6.2,3.4,5.4,2.3\n6.4,2.7,5.3,1.9\n6.9,3.1,5.1,2.3\n5.1,3.7,1.5,0.4\n5.4,3.0,4.5,1.5\n5.2,3.4,1.4,0.2\n4.5,2.3,1.3,0.3\n6.7,3.0,5.2,2.3\n5.7,2.9,4.2,1.3\n6.7,3.0,5.0,1.7\n6.0,3.4,4.5,1.6\n6.1,2.9,4.7,1.4\n5.0,2.3,3.3,1.0\n4.4,3.0,1.3,0.2\n4.9,3.0,1.4,0.2\n6.1,2.6,5.6,1.4\n6.0,2.9,4.5,1.5\n6.7,2.5,5.8,1.8\n4.9,2.5,4.5,1.7\n6.4,3.2,4.5,1.5\n6.1,3.0,4.9,1.8\n'

#### Endpoint - Multiple records from a S3 file

In [54]:
import pandas as pd

In [57]:
infer_ip_s3_uri = "s3://{}/{}/{}".format(
    bucket, prefix, "batch_transform/iris_infer.csv"
)

# payload_df = pd.read_csv(infer_ip_s3_uri)
# payload_df = wr.s3.read_csv(path=infer_ip_s3_uri)
s3_clnt = boto3.client("s3")
obj = s3_clnt.get_object(Bucket=bucket, Key="iris/batch_transform/iris_infer.csv")
payload_df = pd.read_csv(obj["Body"])

csv_buffer = io.StringIO()
payload_df.to_csv(csv_buffer, header=None, index=None)
payload_csv_text = csv_buffer.getvalue()


In [59]:
response = sgmkr_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text
)
print(response["Body"].read().decode())

1.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,2.0


#### Delete the endpoint

In [60]:
sgmkr_clnt = boto3.client("sagemaker")

In [61]:
sgmkr_clnt.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '8a30633c-c555-4db0-8a17-9bd9d4938093',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8a30633c-c555-4db0-8a17-9bd9d4938093',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 18 Oct 2024 01:41:11 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

#### Batch Transform

In [62]:
batch_ip = "s3://{}/{}/{}".format(bucket, prefix, "batch_transform")
batch_op = "s3://{}/{}/{}".format(bucket, prefix, "batch_transform")

In [70]:
print(batch_ip)
print(batch_op)

s3://pat-demo-bkt/iris/batch_transform
s3://pat-demo-bkt/iris/batch_transform


In [71]:
!aws s3 ls {batch_ip}/ --recursive

2024-10-16 17:58:53          0 iris/batch_transform/
2024-10-16 17:59:13        850 iris/batch_transform/iris_infer.csv


In [72]:
transformer = xgb_estimator.transformer(
    instance_count=1, instance_type="ml.m4.xlarge", output_path=batch_op
)

INFO:sagemaker:Creating model with name: iris-xgboost-2024-10-18-02-15-22-272


In [69]:
transformer.transform(
    data=batch_ip, data_type="S3Prefix", content_type="text/csv", split_type="Line"
)
transformer.wait()

Creating transform job with name: iris-xgboost-2023-03-22-08-17-22-760


.................................
Arguments: serve
[2023-03-22 08:22:53 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2023-03-22 08:22:53 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2023-03-22 08:22:53 +0000] [1] [INFO] Using worker: gevent
[2023-03-22 08:22:53 +0000] [21] [INFO] Booting worker with pid: 21
[2023-03-22 08:22:53 +0000] [22] [INFO] Booting worker with pid: 22
Arguments: serve
[2023-03-22 08:22:53 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2023-03-22 08:22:53 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2023-03-22 08:22:53 +0000] [1] [INFO] Using worker: gevent
[2023-03-22 08:22:53 +0000] [21] [INFO] Booting worker with pid: 21
[2023-03-22 08:22:53 +0000] [22] [INFO] Booting worker with pid: 22
[2023-03-22 08:22:53 +0000] [23] [INFO] Booting worker with pid: 23
[2023-03-22 08:22:53 +0000] [24] [INFO] Booting worker with pid: 24
/opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65: MonkeyPatchWarning: Monkey-patching ssl after ssl 

<IPython.core.display.Javascript object>

In [75]:
!aws s3 ls {bucket}/{prefix}/batch_transform/ --recursive

2024-10-16 17:58:53          0 iris/batch_transform/
2024-10-16 17:59:13        850 iris/batch_transform/iris_infer.csv


In [74]:
!aws s3 cp s3://{bucket}/{prefix}/batch_transform/iris_infer.csv.out .

fatal error: An error occurred (404) when calling the HeadObject operation: Key "iris/batch_transform/iris_infer.csv.out" does not exist


In [76]:
!head -n 5 iris_infer.csv.out

2.0
1.0
0.0
1.0
2.0


<IPython.core.display.Javascript object>